In [1]:
import spotipy
import spotipy.oauth2
import spotipy.util as util
import pandas as pd
import numpy as np

In [2]:
import os
from json.decoder import JSONDecodeError
#username = 'rickypan'
#client_id = '23888f5deee6452db5d78bd2d1091da5'
#client_secret = '3913dc71d36b410699ce56ca2b66b9d2'
#redirect_uri = 'http://localhost:8080/callback/'
#scope = None

In [3]:
def get_songs(playlists):
    song_ids = []
    for j in spotify.user_playlist('spotify', playlists)['tracks']['items']:
        song_ids.append(j['track']['id'])
    return(song_ids)

In [4]:
#credentials = spotipy.oauth2.SpotifyClientCredentials(
#        client_id='23888f5deee6452db5d78bd2d1091da5',
#        client_secret='f6ecea7a89d24b02b2c969745b88a9db')

#token = credentials.get_access_token()

In [5]:
username = 'rickypan'
scope = "user-read-currently-playing"
redirect_uri = 'http://localhost/callback'
CLIENT_ID = '23888f5deee6452db5d78bd2d1091da5'
CLIENT_SECRET = 'f6ecea7a89d24b02b2c969745b88a9db'

In [66]:
token = util.prompt_for_user_token(username, scope, CLIENT_ID, CLIENT_SECRET, redirect_uri)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=23888f5deee6452db5d78bd2d1091da5&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2Fcallback&scope=user-read-currently-playing in your browser






In [67]:
spotify = spotipy.Spotify(auth=token)

In [68]:
playlist = 'spotify:user:1221028518:playlist:2YRe7HRKNRvXdJBp9nXFza'
username = '1221028518'

In [9]:
def get_playlist_tracks(username,playlist_id):
    results = spotify.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    return tracks

In [10]:
get_playlist_tracks('1221028518', playlist)[0]['track']['popularity'] #testing

92

In [11]:
get_playlist_tracks('1221028518', playlist)[0]['track']['uri'] #testing

'spotify:track:7qiZfU4dY1lWllzX7mPBI3'

In [12]:
get_playlist_tracks('1221028518', playlist)[0]['track']['name'] #testing

'Shape of You'

In [13]:
get_playlist_tracks('1221028518', playlist)[0]['track']['artists'][0]['name'] #testing

'Ed Sheeran'

In [14]:
playlist_songs = get_playlist_tracks('1221028518', playlist) 

In [15]:
spotify.artist(playlist_songs[2]['track']['artists'][0]['uri'])['genres']

['pop', 'tropical house']

In [16]:
def get_feats(the_playlist, limit_num, offset_num):
    song_uris = []
    for i in range(offset_num, offset_num + limit_num):
        if i >= len(the_playlist):
            break
        curr_uri = the_playlist[i]['track']['uri']
        song_uris.append(curr_uri)
    song_feats = spotify.audio_features(tracks = song_uris)
    return(song_feats)

In [17]:
top_0_100 = get_feats(playlist_songs, limit_num = 100, offset_num = 0)
top_100_200 = get_feats(playlist_songs, limit_num = 100, offset_num = 100)
top_200_300 = get_feats(playlist_songs, limit_num = 100, offset_num = 200)
top_300_400 = get_feats(playlist_songs, limit_num = 100, offset_num = 300)
top_400_500 = get_feats(playlist_songs, limit_num = 100, offset_num = 400)
top_500_600 = get_feats(playlist_songs, limit_num = 100, offset_num = 500)

In [18]:
df_0_100 = pd.DataFrame.from_dict(top_0_100)
df_100_200 = pd.DataFrame.from_dict(top_100_200)
df_200_300 = pd.DataFrame.from_dict(top_200_300)
df_300_400 = pd.DataFrame.from_dict(top_300_400)
df_400_500 = pd.DataFrame.from_dict(top_400_500)
df_500_600 = pd.DataFrame.from_dict(top_500_600)

In [75]:
dat_all = pd.concat([df_0_100, df_100_200, df_200_300, df_300_400, df_400_500, df_500_600])

In [76]:
pop_lst = []
artist_name_lst = []
song_name_lst = []
genre_lst = []

for i in range(0, len(dat_all)):
    curr_pop = playlist_songs[i]['track']['popularity']
    curr_song = playlist_songs[i]['track']['name']
    curr_name = playlist_songs[i]['track']['artists'][0]['name']
    curr_genre = spotify.artist(playlist_songs[i]['track']['artists'][0]['uri'])['genres']
    
    pop_lst.append(curr_pop)
    song_name_lst.append(curr_song)
    artist_name_lst.append(curr_name)
    genre_lst.append(curr_genre)

    
dat_all['popularity'] = pop_lst
dat_all['song'] = song_name_lst
dat_all['artist'] = artist_name_lst
dat_all['genre'] = genre_lst

In [77]:
dat_copy = dat_all.copy()

In [83]:
dat_copy.head(3)

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,tempo,time_signature,track_href,type,uri,valence,popularity,song,artist,genre
0,0.58100,https://api.spotify.com/v1/audio-analysis/7qiZ...,0.825,233713,0.652,7qiZfU4dY1lWllzX7mPBI3,0.00000,1,0.0931,-3.183,...,95.977,4,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,audio_features,spotify:track:7qiZfU4dY1lWllzX7mPBI3,0.931,92,Shape of You,Ed Sheeran,[pop]
1,0.00902,https://api.spotify.com/v1/audio-analysis/12VW...,0.785,173987,0.617,12VWzyPDBCc8fqeWCAfNwR,0.00246,1,0.3510,-5.871,...,103.981,4,https://api.spotify.com/v1/tracks/12VWzyPDBCc8...,audio_features,spotify:track:12VWzyPDBCc8fqeWCAfNwR,0.382,74,One Dance,Drake,"[canadian pop, hip hop, pop rap, rap]"
2,0.41500,https://api.spotify.com/v1/audio-analysis/7crM...,0.736,245507,0.541,7crMiinWx373rNBZBaVske,0.00000,8,0.1100,-5.597,...,94.962,4,https://api.spotify.com/v1/tracks/7crMiinWx373...,audio_features,spotify:track:7crMiinWx373rNBZBaVske,0.662,65,Closer,The Chainsmokers,"[pop, tropical house]"


In [84]:
dat_all = dat_copy

In [86]:
for i in range(0, len(dat_all)):
    
    if (not dat_all['genre'].iloc[i]) or (dat_all['genre'].iloc[i] == []) or (dat_all['genre'].iloc[i] == None): #if empty list then missing genre 
        dat_all['genre'].iloc[i] = 'Unknown'
        
    elif ('latino' in dat_all['genre'].iloc[i]) or ('trap latino' in dat_all['genre'].iloc[i]) or ('latin' in dat_all['genre'].iloc[i]) or ('hispanic' in dat_all['genre'].iloc[i]) or ('latin pop' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'latino'  
    elif ('french rock' in dat_all['genre'].iloc[i]) or ('french indie pop' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'French'
    elif ('k-hop' in dat_all['genre'].iloc[i]) or ('k-pop' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'Korean'
    elif 'german pop' in dat_all['genre'].iloc[i]:
        dat_all['genre'].iloc[i] = 'German'
    elif ('edm' in dat_all['genre'].iloc[i]) or ('electro' in dat_all['genre'].iloc[i]) or ('electronic' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'electronic'  
    elif ('deep house' in dat_all['genre'].iloc[i]) or ('swedish house' in dat_all['genre'].iloc[i]) or ('progressive electro house' in dat_all['genre'].iloc[i]) or ('deep tropical house' in dat_all['genre'].iloc[i]) or ('tropical house' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'house'
    elif ('folk-pop' in dat_all['genre'].iloc[i]) or ('folk' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'folk'  
    elif ('indie' in dat_all['genre'].iloc[i]) or ('indie pop' in dat_all['genre'].iloc[i]) or ('indie optimism' in dat_all['genre'].iloc[i]) or ('indietronica' in dat_all['genre'].iloc[i]) or ('indie folk' in dat_all['genre'].iloc[i]) or ('la indie' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'indie'
    elif ('alternative metal' in dat_all['genre'].iloc[i]) or ('nu metal' in dat_all['genre'].iloc[i]) or ('rap metal' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'metal'
    elif 'hip hop' in dat_all['genre'].iloc[i]:
        dat_all['genre'].iloc[i] = 'hip hop'
    elif ('rap' in dat_all['genre'].iloc[i]) or ('pop rap' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'rap'
    elif ('pop punk' in dat_all['genre'].iloc[i]) or ('punk' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'punk'
    elif ('rock' in dat_all['genre'].iloc[i]) or ('modern rock' in dat_all['genre'].iloc[i]) or ('australian alternative rock' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'rock'
    elif ('alternative r&b' in dat_all['genre'].iloc[i]) or ('r&b' in dat_all['genre'].iloc[i]) or ('indie r&b' in dat_all['genre'].iloc[i]):
        dat_all['genre'].iloc[i] = 'r&b'
    elif 'emo' in dat_all['genre'].iloc[i]:
        dat_all['genre'].iloc[i] = 'emo'
        
    elif 'pop' or 'pop rock' in dat_all['genre'].iloc[i]:
        dat_all['genre'].iloc[i] = 'pop' #big catch all
    
    elif len(dat_all['genre'].iloc[i]) == 1:
        dat_all['genre'].iloc[i] = dat_all['genre'].iloc[i][0] #if 1 genre keep genre

/home/ricky/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [87]:
np.unique(dat_all.genre)

array(['French', 'German', 'Korean', 'Unknown', 'electronic', 'folk',
       'hip hop', 'house', 'indie', 'latino', 'metal', 'pop', 'punk',
       'r&b', 'rap', 'rock'], dtype=object)

In [93]:
#only run this to output csv
dat_all.to_csv('top_all_time.csv')